## Imports

In [1]:
# Add project root to Python path ONLY FOR NOTEBOOK!!!
import sys
from pathlib import Path

project_root = (
    Path(__file__).resolve().parent if "__file__" in globals() else Path().resolve()
)
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

In [2]:
from neo4j import GraphDatabase
from qdrant_client import QdrantClient, models
from dotenv import load_dotenv
from pydantic import BaseModel
from openai import OpenAI
from collections import defaultdict
from neo4j_graphrag.retrievers import QdrantNeo4jRetriever
import uuid
import os

from config.datasets import GraphComponents, Single

In [3]:
# Load environment variables
load_dotenv()

True

---

## Setting Up Environment Variables

In [4]:
# Get credentials from environment variables (derived from docker-compose config)
qdrant_port = os.getenv("QDRANT_HTTP_PORT", "6333")
qdrant_host = os.getenv("QDRANT_URL", "http://localhost")
qdrant_url = f"{qdrant_host}:{qdrant_port}"


# Parse NEO4J_AUTH (format: username/password)
neo4j_auth = os.getenv("NEO4J_AUTH", "neo4j/password")
neo4j_username, neo4j_password = neo4j_auth.split("/", 1)
neo4j_bolt_port = os.getenv("NEO4J_BOLT_PORT")
neo4j_url = os.getenv("NEO4J_URL")
neo4j_uri = f"{neo4j_url}:{neo4j_bolt_port}"
collection_name = os.getenv("QDRANT_COLLECTION_NAME")

# LLM:
llm_model = os.getenv("LLM_MODEL")
llm_api_key = os.getenv("LLM_API_KEY")

# Data and Other:
data_folder = os.getenv("RAW_DATA_FOLDER")

In [5]:
# Print out all envs:
print(f"QDRANT PORT: {qdrant_port}")
print(f"QDRANT HOST: {qdrant_host}")
print(f"QDRANT URL: {qdrant_url}")
print(f"Collection Name: {collection_name}")
print(f"NEO4J URI: {neo4j_uri}")
print(f"NEO4J USERNAME: {neo4j_username}")
print(f"NEO4J PASSWORD: {neo4j_password}")
print(f"LLM MODEL: {llm_model}")
print(f"Data Folder: {data_folder}")


QDRANT PORT: 6333
QDRANT HOST: http://localhost
QDRANT URL: http://localhost:6333
Collection Name: QdrantRagCollection
NEO4J URI: bolt://localhost:7687
NEO4J USERNAME: neo4j
NEO4J PASSWORD: password
LLM MODEL: perplexity/sonar-pro
Data Folder: ./raw_data/


---

## 0.Initialization of Neo4j and Qdrant Clients

### Initiate Qdrant and Neo4j 

In [6]:
neo4j_driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_username, neo4j_password))

qdrant_client = QdrantClient(
    url=qdrant_url,
)

### Creating Qdrant Collection

* Creating neo4j and qdrant clients
* Creating qdrant collection


In [7]:
from rag_handler.qdrant_orchestrator import QdrantOrchestrator

qdrant_orchestrator = QdrantOrchestrator(
    qdrant_url=qdrant_url, collection_name=collection_name, qdrant_key=None
)

qdrant_orchestrator.create_collection()


Collection 'QdrantRagCollection' not found. Creating it now...
Collection 'QdrantRagCollection' created successfully.


# 1. Ingestion - read files

In [8]:
from ingestion.file_reader import FileReader
from ingestion.chunker_embedder import ChunkerEmbedder

### Reading Files

In [9]:
file_reader = FileReader(folder_path=data_folder)
files = file_reader.read_files()
chunk_embedder = ChunkerEmbedder(
    all_files=files,
    chunk_size=os.getenv("CHUNK_SIZE"),
    chunk_overlap=os.getenv("CHUNK_OVERLAP"),
)


# files = {
#     "pdf": ["/home/nir/projects/graph_rag/raw_data/Nir_Potasman_CV.pdf"],
#     "text": [
#         "/home/nir/projects/graph_rag/raw_data/raw_text.txt",
#         "/home/nir/projects/graph_rag/raw_data/short_story.txt",
#     ],
#     "markdown": ["/home/nir/projects/graph_rag/raw_data/raw_text.md"],
#     "image": [],
# }


### Chunking Files
#### For now - I will only chunk the PDF.

In [10]:
chunked_pdf = chunk_embedder.chunk_pdf()

2025-11-25 14:23:47,716 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-11-25 14:23:47,759 - INFO - Going to convert document batch...
2025-11-25 14:23:47,760 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 44ae89a68fc272bc7889292e9b5a1bad
2025-11-25 14:23:47,774 - INFO - Loading plugin 'docling_defaults'
2025-11-25 14:23:47,777 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-11-25 14:23:47,783 - INFO - Loading plugin 'docling_defaults'
2025-11-25 14:23:47,786 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2025-11-25 14:23:48,049 - INFO - Accelerator device: 'cpu'
[INFO] 2025-11-25 14:23:48,065 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-11-25 14:23:48,072 [RapidOCR] download_file.py:60: File exists and is valid: /home/nir/projects/graph_rag/.venv/lib/python3.13/site-packages/rapidocr/models/ch_PP-OCRv4_det_infer.onnx
[INFO] 2025-11-25 14:23:48,073 [RapidOCR] ma

In [11]:
chunked_pdf[0]

{'file': 'Nir_Potasman_CV.pdf',
 'chunks': ['Results-driven  Data  Scientist  leveraging  advanced  analytics,  with  a  background  in biology,  medicine,  machine  learning,  and  statistical  modeling  to  drive  data-informed decision-making.  Skilled  in  extracting  actionable  insights  from  complex  datasets  and communicating findings to both technical  and  non-technical  stakeholders.  Proficient  in Linux,  C,  AWS,  Docker,  Python,  NoSQL,  SQL.  Passionate  about  solving  challenging problems and delivering innovative solutions that drive business growth and operational efficiency.',
  '- Machine Learning & AI\n- Statistical Analysis\n- Data Visualization\n- LLMs',
  '- Bachelor of Science in Biology & Life-Science Tel-Aviv University 2015-2018\n- Bachelor of Medical Sciences Tel-Aviv University 2020-2022\n- Doctor of Medicine\nTel-Aviv University 2020 - Pause',
  '2025 - Present - AI Engineer at Finubit',
  '- Architected and implemented a production-grade, multi-agen

In [12]:
len(chunked_pdf[0]["chunks"])
# The PDF has been chunked to 15 pieces.

15

## 2. Extracting Nodes and Relationships

In [13]:
from ingestion.orchestration import Orchestrator


In [14]:
orchestrator = Orchestrator(llm_model, llm_api_key)

In [15]:
nodes, relationships, chunk_node_mapping = orchestrator.extract_graph_components(
    chunked_pdf
)

14:23:51 - LiteLLM:INFO: utils.py:3427 - 
LiteLLM completion() model= sonar-pro; provider = perplexity
2025-11-25 14:23:51,573 - INFO - 
LiteLLM completion() model= sonar-pro; provider = perplexity
14:23:58 - LiteLLM:INFO: utils.py:1307 - Wrapper: Completed Call, calling success_handler
2025-11-25 14:23:58,488 - INFO - Wrapper: Completed Call, calling success_handler
14:23:58 - LiteLLM:INFO: utils.py:3427 - 
LiteLLM completion() model= sonar-pro; provider = perplexity
2025-11-25 14:23:58,494 - INFO - 
LiteLLM completion() model= sonar-pro; provider = perplexity
14:24:02 - LiteLLM:INFO: utils.py:1307 - Wrapper: Completed Call, calling success_handler
2025-11-25 14:24:02,127 - INFO - Wrapper: Completed Call, calling success_handler
14:24:02 - LiteLLM:INFO: utils.py:3427 - 
LiteLLM completion() model= sonar-pro; provider = perplexity
2025-11-25 14:24:02,132 - INFO - 
LiteLLM completion() model= sonar-pro; provider = perplexity
14:24:07 - LiteLLM:INFO: utils.py:1307 - Wrapper: Completed Ca

---

## 3. Ingest nodes and relationships into Neo4j

In [16]:
from rag_handler.neo4j_orchestrator import Neo4jOrchestrator

neo4_orchestrator = Neo4jOrchestrator(
    neo4j_url=neo4j_uri, auth=(neo4j_username, neo4j_password)
)
neo4_orchestrator.ingest_to_neo4j(nodes, relationships, chunk_node_mapping)

2025-11-25 14:25:31,106 - INFO - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Statement.CartesianProduct} {category: PERFORMANCE} {title: This query builds a cartesian product between disconnected patterns.} {description: If a part of a query contains multiple disconnected patterns, this will build a cartesian product between all those parts. This may produce a large amount of data and slow down query processing. While occasionally intended, it may often be possible to reformulate the query that avoids the use of this cross product, perhaps by adding a relationship between the different parts or by using OPTIONAL MATCH (identifier is: (e))} {position: line: 1, column: 1, offset: 0} for query: 'MATCH (c:Chunk {id: $chunk_id}), (e:Entity {id: $entity_id}) CREATE (c)-[:MENTIONS]->(e)'
2025-11-25 14:25:31,121 - INFO - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Statement.CartesianProduct} {ca

{'Data Scientist': '86d483fe-fe01-4c68-83b2-55495d0faf9f',
 'advanced analytics': '3fa88947-e299-4d07-ad98-9eeca7205cf1',
 'biology': '07cb65f2-1e3c-4343-8699-ea50b9ef7717',
 'medicine': 'cc144264-8e24-44b7-afe6-712bccae82cc',
 'machine learning': '3ca4c6ce-1ac9-48dc-a144-6cb6d2326154',
 'statistical modeling': '3fe1f1c5-8d32-4b8b-a679-b71053f410da',
 'data-informed decision-making': '1610c0d8-b1db-458d-baae-d65229fa4159',
 'complex datasets': '099142d0-67c1-491f-8a8b-c9a41414b093',
 'technical stakeholders': 'a732c1ae-10cc-42b7-9391-e0cc4a735a61',
 'non-technical stakeholders': '018688ae-8fcd-453a-a00c-1c7a320a329b',
 'Linux': '1fe850f5-c59a-4f26-a058-ad3f2bbf7a7a',
 'C': 'b707f558-48ab-43dd-a461-1b0b65752f69',
 'AWS': '1c3e821a-bc13-4844-a1ea-5c2eb0896276',
 'Docker': 'dabd7f00-88a0-40d0-a847-a427ad8f5723',
 'Python': 'ed332136-c3a1-4ac5-a037-29a08285206e',
 'NoSQL': '807ab9bf-bd33-422c-a82f-19c54c7523fe',
 'SQL': '4f7310b0-1395-4fc8-aa86-218ce14e4cc9',
 'challenging problems': 'ebd7

---

## 4. Embed the data

In [17]:
embedded_data = chunk_embedder.embed_chunks(chunked_pdf)

14:25:49 - LiteLLM:INFO: utils.py:1307 - Wrapper: Completed Call, calling success_handler
2025-11-25 14:25:49,331 - INFO - Wrapper: Completed Call, calling success_handler


embedded_data is a dictionary INSIDE A LIST (since there may be multiple files) that looks like this:
```
{
    "source_file" : <file_name.pdf>,
    "chunks" : ['chunk_1', 'chunk_2', ...],
    "embeddings" : [embedding_1, embedding_2, ...]
}
```
* number of embeddings as the number of chunks.
* if Nir_CV.pdf has 15 chunks, embeddings will look like this:
```
[[-1.001, -3.512, ...], [0.23541, 1.2345, ...], ... , embedding_15]
```
---

ULTIMATELY, embedded_data looks like this:
```
[
    {
        "source_file" : <file_name.pdf>,
        "chunks" : ['chunk_1', 'chunk_2', ...],
        "embeddings" : [embedding_1, embedding_2, ...]
    },
    {
        "source_file" : <another_file.pdf>,
        "chunks" : ['chunk_1', 'chunk_2', ...],
        "embeddings" : [embedding_1, embedding_2, ...]
    },
    ...
]
```

## 5. Ingest to Qdrant

In [ ]:
# ingesting embedded data to qdrant:
qdrant_orchestrator.ingest_to_qdrant(collection_name, embedded_data, chunk_node_mapping)

---
## Embedding Tester + Query Embedding

In [19]:
user_query: str = "who is Nir Potasman?"


In [20]:
embedded_user_query = chunk_embedder.embedding_text(user_query)

14:25:49 - LiteLLM:INFO: utils.py:1307 - Wrapper: Completed Call, calling success_handler
2025-11-25 14:25:49,817 - INFO - Wrapper: Completed Call, calling success_handler


---
## Testing Retriever

In [37]:
from answering_agent import retriever_search

In [38]:
retriever_result = retriever_search(
    neo4j_driver=neo4j_driver,
    qdrant_client=qdrant_client,
    collection_name=collection_name,
    embedded_query=embedded_user_query,
)

In [39]:
retriever_result

RetrieverResult(items=[RetrieverResultItem(content="<Record node=<Node element_id='4:e110a383-9bb0-449b-b30c-330a9125d666:219' labels=frozenset({'Chunk'}) properties={'chunk_index': 6, 'text': '+972-5222-73418\\nnirpot@gmail.com', 'id': 'f90a666b-f198-4ec1-8abd-61a1c3c351be', 'source_file': 'Nir_Potasman_CV.pdf'}> score=0.51536036>", metadata=None), RetrieverResultItem(content="<Record node=<Node element_id='4:e110a383-9bb0-449b-b30c-330a9125d666:221' labels=frozenset({'Chunk'}) properties={'chunk_index': 8, 'text': '- USA\\n- Israeli', 'id': '83208d33-1b37-4c6c-86a5-39167c36e0aa', 'source_file': 'Nir_Potasman_CV.pdf'}> score=0.40461433>", metadata=None), RetrieverResultItem(content="<Record node=<Node element_id='4:e110a383-9bb0-449b-b30c-330a9125d666:220' labels=frozenset({'Chunk'}) properties={'chunk_index': 7, 'text': '- English - Native\\n- Hebrew - Native', 'id': '94fec78c-e721-4f24-b469-a6cb4cf3a6dd', 'source_file': 'Nir_Potasman_CV.pdf'}> score=0.37882593>", metadata=None), Ret

In [40]:
chunk_ids = [
    item.content.split("'id': '")[1].split("'")[0] for item in retriever_result.items
]


In [41]:
chunk_ids

['f90a666b-f198-4ec1-8abd-61a1c3c351be',
 '83208d33-1b37-4c6c-86a5-39167c36e0aa',
 '94fec78c-e721-4f24-b469-a6cb4cf3a6dd',
 '37859f93-9238-45eb-b38f-e0672ff75f07',
 '9ba73eed-c6a7-4de5-94e0-56ad6cd43510']

In [ ]:
def fetch_related_graph(neo4j_client, chunk_ids):
    query = """
    // Start from Chunks, find their Entities
    MATCH (c:Chunk)-[:MENTIONS]->(e:Entity)
    WHERE c.id IN $chunk_ids
    
    // Get entities and their 1-hop relationships
    OPTIONAL MATCH (e)-[r]-(related:Entity)
    RETURN e, r, related
    """
    with neo4j_client.session() as session:
        result = session.run(query, chunk_ids=chunk_ids)
        subgraph = []
        for record in result:
            if record["r"] and record["related"]:
                subgraph.append(
                    {
                        "entity": record["e"],
                        "relationship": record["r"],
                        "related_node": record["related"],
                    }
                )
        return subgraph

In [43]:
subgraph_answers = fetch_related_graph(neo4j_client=neo4j_driver, chunk_ids=entity_ids)

In [44]:
subgraph_answers

[{'entity': <Node element_id='4:e110a383-9bb0-449b-b30c-330a9125d666:26' labels=frozenset({'Entity'}) properties={'name': 'Bachelor of Science in Biology & Life-Science', 'id': '07805672-9e11-4791-aadd-5d0ac07bec32'}>,
  'relationship': <Relationship element_id='5:e110a383-9bb0-449b-b30c-330a9125d666:261' nodes=(<Node element_id='4:e110a383-9bb0-449b-b30c-330a9125d666:26' labels=frozenset({'Entity'}) properties={'name': 'Bachelor of Science in Biology & Life-Science', 'id': '07805672-9e11-4791-aadd-5d0ac07bec32'}>, <Node element_id='4:e110a383-9bb0-449b-b30c-330a9125d666:28' labels=frozenset({'Entity'}) properties={'name': '2015-2018', 'id': 'b529b305-4084-4f0f-8bf2-556e6a2a1226'}>) type='spanning_years' properties={'original_type': 'spanning_years', 'source_file': 'Nir_Potasman_CV.pdf'}>,
  'related_node': <Node element_id='4:e110a383-9bb0-449b-b30c-330a9125d666:28' labels=frozenset({'Entity'}) properties={'name': '2015-2018', 'id': 'b529b305-4084-4f0f-8bf2-556e6a2a1226'}>},
 {'entity

In [45]:
def format_graph_context(subgraph):
    nodes = set()
    edges = []

    for entry in subgraph:
        entity = entry["entity"]
        related = entry["related_node"]
        relationship = entry["relationship"]

        nodes.add(entity["name"])
        nodes.add(related["name"])

        edges.append(f"{entity['name']} {relationship['type']} {related['name']}")

    return {"nodes": list(nodes), "edges": edges}

In [49]:
graph_context = format_graph_context(subgraph=subgraph_answers)

In [50]:
graph_context

{'nodes': ['feature engineering',
  'Doctor of Medicine',
  'GCP',
  'Infinity Labs',
  '2020-2022',
  'Previous coding experience',
  'Python',
  'Tel-Aviv University',
  'exploratory analysis',
  'Portfolio',
  'Hebrew',
  'Professionals transitioning into AI roles',
  'Bachelor of Science in Biology & Life-Science',
  'GCP Model Garden',
  'NLP',
  'Finubit',
  'Software Engineering',
  '2020-Pause',
  'Predictive Modeling',
  'Machine learning frameworks',
  'Data Scientist',
  'High-impact careers in tech',
  'teams',
  'Person',
  'English',
  '+972-5222-73418',
  'Artificial Intelligence',
  'Web Scraping',
  'AI/ML solutions',
  'Paused as of November 25, 2025',
  'Flexible payment plans',
  'Infinity Labs training programs',
  'Data Science',
  'document processing',
  'nirpot@gmail.com',
  'Visionaries aiming to build AI-powered products',
  'A/B Testing',
  'Hands-on experience',
  'TensorFlow',
  'Beginners with no coding background',
  'Real-time code review and feedback',

In [60]:
def extract_chunk_text_from_retriever_results(retriever_result):
    """Extract the actual text from chunks returned by retriever"""
    chunks = []
    for item in retriever_result.items:
        # Parse the chunk text from the Neo4j node properties
        content = item.content
        # Extract text property
        text_start = content.find("'text': '") + len("'text': '")
        text_end = content.find("'", text_start)
        chunk_text = content[text_start:text_end]
        
        # Also get the score for ranking
        score_start = content.find("score=") + len("score=")
        score_end = content.find(">", score_start)
        score = float(content[score_start:score_end])
        
        chunks.append({
            "text": chunk_text,
            "score": score
        })
    return chunks

In [61]:
chunk_texts = extract_chunk_text_from_retriever_results(retriever_result)


In [62]:
from litellm import completion


In [63]:
def graphRAG_run(chunk_texts, graph_context, user_query):
    """
    Combine semantic chunks with graph knowledge
    
    Args:
        chunk_texts: List of relevant text chunks from Qdrant
        graph_context: Dict with nodes and edges from Neo4j
        user_query: User's question
    """
    # Format relevant chunks
    chunks_str = "\n\n".join([
        f"Chunk {i+1} (relevance: {chunk['score']:.2f}):\n{chunk['text']}"
        for i, chunk in enumerate(chunk_texts)
    ])
    
    # Format graph context
    nodes_str = ", ".join(graph_context["nodes"])
    edges_str = "; ".join(graph_context["edges"])
    
    prompt = f"""
    You are an intelligent assistant. Use both the semantically relevant text chunks 
    and the knowledge graph to provide an accurate, comprehensive answer.
    
    === RELEVANT TEXT CHUNKS (from vector search) ===
    {chunks_str}
    
    === KNOWLEDGE GRAPH (entities and relationships) ===
    Nodes: {nodes_str}
    
    Edges: {edges_str}
    
    === USER QUESTION ===
    {user_query}
    
    === INSTRUCTIONS ===
    1. First, identify relevant information from the text chunks (direct quotes)
    2. Then, enrich your answer using the knowledge graph relationships
    3. Provide a comprehensive answer that combines both sources
    4. Cite specific chunks when using direct information
    """
    
    try:
        response = completion(
            model=llm_model,
            api_key=llm_api_key,
            messages=[
                {
                    "role": "system",
                    "content": "Provide a thorough answer using both the text chunks and knowledge graph.",
                },
                {"role": "user", "content": prompt},
            ],
        )
        return response.choices[0].message
    
    except Exception as e:
        return f"Error querying LLM: {str(e)}"

In [64]:
chunk_texts = extract_chunk_text_from_retriever_results(retriever_result)


In [65]:
final_answer = graphRAG_run(
    chunk_texts=chunk_texts,
    graph_context=graph_context, 
    user_query=user_query
)

14:51:49 - LiteLLM:INFO: utils.py:3427 - 
LiteLLM completion() model= sonar-pro; provider = perplexity
2025-11-25 14:51:49,702 - INFO - 
LiteLLM completion() model= sonar-pro; provider = perplexity
14:52:05 - LiteLLM:INFO: utils.py:1307 - Wrapper: Completed Call, calling success_handler
2025-11-25 14:52:05,023 - INFO - Wrapper: Completed Call, calling success_handler


In [66]:
final_answer

Message(content='Nir Potasman is an Israeli and American professional with a strong academic and technical background in life sciences, medicine, and artificial intelligence, currently paused in medical studies and possessing expertise in advanced AI/ML technologies.\n\nKey details drawn from the text chunks and knowledge graph:\n\n- **Contact Information**: "+972-5222-73418" and "nirpot@gmail.com"[1].\n- **Nationality**: Holds ties to both **Israel** and the **USA**[2].\n- **Languages**: **Native** proficiency in **English** and **Hebrew**[3].\n\n### Academic Background\n- **Bachelor of Science in Biology & Life-Science** from **Tel-Aviv University** (2015–2018)[5].\n- **Bachelor of Medical Sciences** from **Tel-Aviv University** (2020–2022)[5].\n- **Doctor of Medicine** studies at Tel-Aviv University, **paused as of November 25, 2025** (began 2020)[5].\n  - The knowledge graph confirms that the Doctor of Medicine program is currently paused and shows relationships linking these degre